In [6]:
import numpy as np
import pandas as pd
import datetime
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb

import sys, os, gc, types
import time
from subprocess import check_output

import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
root_paths = [
    "/data/kaggle-wikipedia/data/",
    "/Users/jiayou/Dropbox/JuanCode/Kaggle/Wikipedia/data/",
    "/Users/jiayou/Dropbox/Documents/JuanCode/Kaggle/Wikipedia/data/"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
print(check_output(["ls", root]).decode("utf8"))

date_df.csv
key_1.csv
median_49.csv
page_df.csv
page_ohe.csv
sample_submission_1.csv
spider.txt
train_1.csv



In [ ]:
train = pd.read_csv(root + 'train_1.csv')
train.fillna(0, inplace = True)
date_df = pd.read_csv(root + 'date_df.csv')
page_df = pd.read_csv(root + 'page_ohe.csv')
median_49 = pd.read_csv(root + 'median_49.csv')

In [ ]:
train_df = train.melt(id_vars=['Page'], var_name='date')

# construct ABT

In [ ]:
train_df = train_df.merge(page_df, how='left', on='Page')
train_df = train_df.merge(date_df.drop('date_str', axis = 1), how='left', on='date')

In [ ]:
val_days = 62
for i in range(-val_days, 0):
    median_49.iloc[:, i] = median_49.iloc[:, -val_days-1].reshape((len(median_49),1))

In [ ]:
median_df = median_49.melt(id_vars=['Page'], var_name='date')
median_df.rename(columns={'value':'median'}, inplace = True)
train_df = train_df.merge(median_df, how='left', on=['Page','date'])

In [ ]:
train_df['isval'] = (train_df.dayofyear > 366 - val_days) & (train_df.year == 2016)

In [ ]:
train_df.head()

In [ ]:
del median_df, page_df, date_df, train
gc.collect()

# Preparing data and hyperparams

In [ ]:
name = 'v0-r0'
down_sample = None
num_searches = 1
boosting_rounds = 10000
stopping_rounds = 10

In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l1'},
    'num_leaves': 128,
#     'min_sum_hessian_in_leaf': 20,
    'max_depth': 12,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.85,
    'bagging_freq': 3,
    'verbose': 1,
    'device' : 'gpu'
}

In [ ]:
train = train_df[train_df.isval == False]
val = train_df[train_df.isval == True]

In [ ]:
lgb_train = lgb.Dataset(train.drop(['value', 'isval', 'Page', 'date'], axis = 1), train.value)
lgb_eval = lgb.Dataset(val.drop(['value', 'isval', 'Page', 'date'], axis = 1), val.value, reference=lgb_train)

gc.collect()

In [ ]:
results = []
for i in range(num_searches):
    print('Start LightGBM training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=boosting_rounds,
                    valid_sets=[lgb_train, lgb_eval],
#                   categorical_feature=[],
                    early_stopping_rounds=stopping_rounds)

    print('Save model...')
    # save model to file
    gbm.save_model('model.{}.txt'.format(name))

    print('Plot feature importances...')
    ax = lgb.plot_importance(gbm, max_num_features=10, importance_type='gain')
    plt.show()